This is a cover letter builder using agentic ai


In [87]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import json

In [88]:
load_dotenv(override=True)


True

In [89]:
openai = OpenAI()

In [90]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [91]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [92]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [93]:
name = "Sviatoslav Rutkovskyi"


In [94]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover lettter. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
"""


In [95]:
evaluator_system_prompt = f"""
You are an evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. 
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [96]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [97]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [98]:
def update_system_prompt(cover_letter, feedback):
        return system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """

In [99]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.chat.completions.create(model=evaluator_model, messages=messages)
    content = response.choices[0].message.content
    data = json.loads(content)
    return Evaluation(**data)

In [100]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [101]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [102]:
def requestLetter(job_posting, history):
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [103]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Failed evaluation - retrying


## Cover Letter:
Dear Hiring Manager,

I am writing to express my enthusiasm for the Software QA Engineer role at WireWheel. My Bachelor of Science in Computer Science from the University of Washington Tacoma, coupled with significant experience developing robust applications and a strong understanding of software engineering practices, make me well-suited to contribute to WireWheel's mission of enhancing privacy and data protection through innovative software solutions.

Throughout my academic journey and varied projects, I have demonstrated a commitment to producing high-quality code and ensuring software reliability. The Asymmetric Encryption App I implemented, featuring advanced cryptographic algorithms, reflects my dedication to creating secure software solutions, an ethos that aligns closely with WireWheel’s focus on safeguarding user data. Leading projects like the Bookkeeping Web App has honed my skills in fostering collaboration and problem-solving, pivotal to thriving in your dynamic, fast-paced start-up environment. I am eager to bring my proficiency in React, Node.js, and various testing frameworks to your team to enhance your automated quality assurance suite and deliver reliable solutions.

My background as a sergeant in the US Army Reserves has endowed me with excellent leadership capabilities, the capacity to work adeptly under pressure, and a strong foundation in task management. These skills, complemented by my technical proficiencies in API development and experience with tools like Docker and Postman, will enable me to seamlessly integrate with your team, contribute to the creation and improvement of a test automation framework, and enhance overall product quality. I am excited about the prospect of collaborating with WireWheel’s distinguished team to innovate and develop solutions that not only support your strategic goals but also exceed customer expectations.

Thank you for considering my application. I look forward to bringing my skills and passion for software quality to WireWheel.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
The letter is professional in tone, follows the three-paragraph structure, and demonstrates strong academic and leadership credentials. However, it reads more like a general software engineering pitch than a focused QA application. To make it truly tailored to the Software QA Engineer role at WireWheel, you should explicitly reference your hands-on experience with test automation tools and frameworks (for example Playwright and TypeScript), your familiarity with continuous integration systems (such as Jenkins or CircleCI), and specific examples of how you have written and maintained automated tests to catch bugs. Additionally, you could strengthen your opening by connecting more directly to WireWheel’s mission and culture, and conclude with a clearer statement of how your QA skills will drive quality in their privacy-focused platform.

Failed evaluation - retrying


...

## Feedback:
While your cover letter is professional and clearly outlines your background, it reads as a generic template rather than a deeply tailored response to WireWheel’s specific needs. You do a good job summarizing your academic work and leadership experience, but you never mention the exact tools and responsibilities called out in the job description—Playwright with TypeScript, API test automation frameworks, findings bugs through automated suites, and continuous integration processes. Your opening paragraph is polite but lacks a hook that demonstrates your passion for data privacy or what draws you personally to WireWheel. To improve: 1) Open with a specific achievement or insight about WireWheel’s mission in privacy and how it resonates with you. 2) In your second paragraph, cite a project where you built automated tests (ideally with Playwright or a similar tool) or integrated tests into CI/CD pipelines. 3) Close by reaffirming your enthusiasm for WireWheel’s culture of innovation, and suggest next steps or an action (e.g., discussing how you would build their test suite). Making these changes will show you’ve read the posting closely and are uniquely qualified for their QA role.

Failed evaluation - retrying


## Cover Letter:
Dear Hiring Manager,

I am excited to apply for the Software QA Engineer position at WireWheel. With a Bachelor of Science in Computer Science from the University of Washington Tacoma and a minor in Mathematics, I am passionate about advancing data privacy and protection technologies, which aligns closely with WireWheel's mission. Your innovative approach to leveraging cloud infrastructure and machine learning for safeguarding personal data greatly resonates with my career aspirations, and I am eager to contribute to the company's growth and success in this realm.

Throughout my academic and project experiences, I have demonstrated my ability to build quality software through test automation and API development. My role as a team leader on the Bookkeeping Web App project involved creating secure server-side APIs using Node.js, which required implementing robust testing frameworks to ensure consistent application performance. While I have not specifically worked with Playwright, my proficiency in TypeScript and frameworks like React and Node.js positions me strongly for developing and enhancing a web test automation suite. Additionally, my experience in machine learning projects has equipped me with the analytical skills necessary for identifying and resolving bugs, while my foundational understanding of continuous integration systems enables me to adapt quickly to WireWheel's QA processes.

The opportunity to join WireWheel's dynamic, diverse, and driven team is especially compelling. I am drawn to the way WireWheel invests in individual career development and fosters a culture of innovation. I am enthusiastic about leveraging my skills in collaboration and problem-solving to contribute to your team’s efforts in creating superior products. I would be excited to further discuss how I can play a role in enhancing WireWheel’s platform and ensuring the highest quality software. Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
While your cover letter follows the three-paragraph structure and avoids extra formatting, it reads as somewhat generic and misses opportunities to really showcase why you and WireWheel are such a great fit. Your opening could be stronger by calling out a particular WireWheel product or challenge that excites you. In the body, you note your experience leading API development and writing testing frameworks, but you don’t give concrete examples of automated test cases you’ve authored or bugs you’ve uncovered. You also admit to no direct Playwright experience; instead of stating that gap outright, describe how you’ve rapidly learned a similar tool or framework and how that skill will transfer. Finally, tie more of your achievements back to the QA role—mention specific test coverage improvements, CI pipelines you’ve configured, or metrics you improved. By sharpening these details and aligning them more closely with the day-to-day responsibilities at WireWheel, your letter will feel more genuine, targeted, and memorable.

Passed evaluation - returning reply
